# Grid extension Examples

You can extend the grid by inheriting from the power-grid-model-ds. In this way you can profit from existing functionality but also add your own.
Here we show how to extend the Grid with output variables in the node and line arrays.

## Adding columns to the Grid arrays

Output variables are not present in the basic Grid as defined in pgm-ds, since these might or might not be useful to specific projects.
To add these as you please you can extend the definitions of the arrays in your own project.


In [ ]:
import numpy as np
from numpy.typing import NDArray

from power_grid_model_ds.arrays import LineArray, NodeArray


class ExtendedNodeArray(NodeArray):
    """Extends the node array with an output value"""

    _defaults = {"u": 0}

    u: NDArray[np.float64]


class ExtendedLineArray(LineArray):
    """Extends the line array with an output value"""

    _defaults = {"i_from": 0}

    i_from: NDArray[np.float64]

## Adding the new arrays to the Grid

With the arrays in place, the recommended path is to let `grid_extension_factory` wire them into a tailor-made `Grid` subclass. This avoids hand-written containers and keeps the example aligned with the implementation shipped in power-grid-model-ds.

In [ ]:
from power_grid_model_ds._core.model.grids.factory import grid_extension_factory

ExtendedGrid, extended_grid = grid_extension_factory(
    name="ExtendedGrid",
    user_arrays={
        "node": ExtendedNodeArray,
        "line": ExtendedLineArray,
    },
)

ExtendedGrid

## Create and inspect a grid instance

The factory call already gives you an initialized grid that is ready for use. Inspecting it confirms the telemetry columns are available.

In [ ]:
extended_grid

list(extended_grid.node.columns), list(extended_grid.line.columns)

## Adding metadata fields

Besides arrays, you can inject simple dataclass fields (strings, numbers, config blobs) via `user_fields`. Provide a `(type, default)` tuple for each entry and the factory wires them into the generated grid class.

In [ ]:
from dataclasses import field

MetaGrid, meta_grid = grid_extension_factory(
    name="MetaGrid",
    user_arrays={
        "node": ExtendedNodeArray,
        "line": ExtendedLineArray,
    },
    user_fields={
        "owner": (str, "DSO"),
        "tags": (list[str], field(default_factory=list)),
    },
)

meta_grid.owner, meta_grid.tags